## 與google drive 連線

In [ ]:
#先安裝授權 google drive 使用
#點擊連結複製認證碼
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [ ]:
#在虛擬平台建立一個 google drive 接口

!mkdir -p drive
!google-drive-ocamlfuse drive

## 安裝kaggle API

In [ ]:
#記得先上傳kaggle.json 到google dirve的最外層
!pip install kaggle

In [ ]:
#匯入kaggle.json
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

In [ ]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
#以上步驟完就可以操作kaggle api ， 簡易使用可參照另一份kaggle installation guide

In [ ]:
# numpy and pandas for data manipulation
import csv
import numpy as np
import pandas as pd

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

#plot
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# load package# load 
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation  import train_test_split
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import xgboost as XGBT
import os


In [ ]:
#下載data
!kaggle competitions download -c esuntoygame2018 -p toygame   #(p是放置的地方，若原本不存在則會建立資料夾)

In [ ]:
df=pd.read_csv("train.csv", encoding="UTF8")
print ("train datastet shape:", df.shape)

In [ ]:
#挑選重要變數
correlations = df.corr()["Class"].sort_values()
correlations=pd.DataFrame(correlations)
select=correlations[abs(correlations.iloc[:,0])>=0.00]

In [ ]:
print (select) 
x_select=select.index.tolist()
x_select.remove("Class")
print (x_select)

In [ ]:
#建立建模ＸＹ
x=df2[x_select] #選一定貢獻度的變數

y=pd.DataFrame(df2['Class'])

In [ ]:
#排除AMOUNT超過3000的樣本   （不排比較好）
#df2=df[df["Amount"]<3000]
df2=df
df2.info()

In [ ]:
#分層抽樣
Target="Class"

y_0len=int(len(y[y[Target]==1].Class.tolist()))
save_0_x=pd.DataFrame.sample(x.iloc[np.where(y[Target]==0)],n=y_0len*100,replace=False,random_state=888)
save_1_x=pd.DataFrame.sample(x.iloc[np.where(y[Target]==1)],n=int(y_0len*1.5),replace=True,random_state=888)

save_0_y=pd.DataFrame.sample(y.iloc[np.where(y[Target]==0)],n=y_0len*100,replace=False,random_state=888)
save_1_y=pd.DataFrame.sample(y.iloc[np.where(y[Target]==1)],n=int(y_0len*1.5),replace=True,random_state=888)

temp_x=pd.DataFrame.append(save_1_x,save_0_x)
temp_y=pd.DataFrame.append(save_1_y,save_0_y)

print (y_0len)

In [ ]:
#我的LGBMClassifier在colab上無法匯入 有點無解 

In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM ; cd LightGBM
import os
os.chdir('LightGBM')
!mkdir build
os.chdir('build')
!apt-get install cmake
!cmake ..
!make -j4

In [ ]:
import LightGbm as lgb
from LightGbm import LGBMClassifier

In [ ]:
LGBM_grid_final = lgb.LGBMClassifier()
clf=LGBM_grid_final

clf.fit(temp_x,temp_y)

test_public=pd.read_csv("test_public.csv", encoding="UTF8")
print ("test datatest shape:", test_public.shape)

print (x_select)
test_model=test_public[x_select]

y_pred=clf.predict(test_model)
print (y_pred)

In [ ]:
# Grid Search
print('Start training...')
estimator = lgb.LGBMClassifier(objective = 'binary', n_estimators = 100, random_state=0)

param_grid = {
    'learning_rate': [0.3],
    'num_leaves': [20],
    'feature_fraction': [0.2,0.3,0.4] ,#[0.4,0.6,0.8] then 0.8,
    'bagging_fraction': [0.005,0.01,0.02],#[0.01,0.02,0.03]then 0.1,
    'max_depth':[6],#5,6],
    'max_bin':[10],#15,20], 
    'lambda_l1':[0.4],#0.5,0.6],
    'lambda_l2':[0.02],#0.03,0.04], 
    'min_split_gain':[0.05],#0.06,0.07],
    'min_child_weight':[3]}#5,7]}#,5,7]

LGBM_grid = GridSearchCV(estimator, param_grid)
LGBM_grid.fit(temp_x, temp_y)
print('Best parameters found by grid search are:', LGBM_grid.best_params_)

# Final Model
evals_result = {} 
print('Start predicting...')
LGBM_grid_final = lgb.LGBMClassifier(objective = 'binary',
                                      #learning_rate = 0.05, 
                                     learning_rate = LGBM_grid.best_params_['learning_rate'],
                                     n_estimators = 100, 
                                     random_state=0,
                              num_leaves = LGBM_grid.best_params_['num_leaves'],
                                feature_fraction = LGBM_grid.best_params_['feature_fraction'], 
                              bagging_fraction = LGBM_grid.best_params_['bagging_fraction'],
                             max_depth = LGBM_grid.best_params_['max_depth'],
                              max_bin = LGBM_grid.best_params_['max_bin'],
                              lambda_l1 = LGBM_grid.best_params_['lambda_l1'],
                              lambda_l2 = LGBM_grid.best_params_['lambda_l2'],
                            min_split_gain = LGBM_grid.best_params_['min_split_gain'],
                            min_child_weight = LGBM_grid.best_params_['min_child_weight'])
LGBM_grid_final_fit = LGBM_grid_final.fit(temp_x, temp_y)
print('Predicting is over')

In [ ]:


#! kaggle competitions submit -c esuntoygame2018 -f submit_to_kaggle.csv -m "Message"  